In [25]:
import time
import numpy as np
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait


driver = webdriver.Chrome(ChromeDriverManager().install())

sainsburys_beer_path = 'https://www.sainsburys.co.uk/shop/gb/groceries/drinks/all-beers'

driver.get(sainsburys_beer_path)

time.sleep(1)

WebDriverWait(driver, timeout=10).until(lambda d: d.find_element(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]'))

driver.find_element(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]').click()

elems = driver.find_elements(By.XPATH,"//a[@href]")

# get links for all beers in the page so we can start visiting beer level information
links = []
for elem in elems:
    pattern = 'https://www.sainsburys.co.uk/shop/gb/groceries/product/details/'
    try:
        attr = elem.get_attribute("href")
        if pattern in attr:
            links.append(attr)
    except:
        pass

def get_abv(abv):
    for t in abv:
        if 'Alcohol' in t.text:
            return t
data = []

# Get %Alcohol and price per liter values 
for l in np.unique(links):
    try:
        driver.get(l) 
        time.sleep(3) 
        x_path_alcohol_volume = '//*[@id="mainPart"]/div[4]/div/p'

        ppl = driver.find_element(By.CLASS_NAME,'pd__cost__per-unit')
        abv_0 = driver.find_elements(By.CLASS_NAME,'nameTextItems')
        abv_1 = get_abv(abv_0)
        entry = [l, ppl.text, abv_1.text]
        data.append(entry)
        print(l, ppl.text, abv_1.text)
    except:
        pass




/var/folders/p6/325r0tsd2cd1jddpt9nd70hm0000gn/T/ipykernel_48388/1563364347.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


https://www.sainsburys.co.uk/shop/gb/groceries/product/details/all-beers/badger-fursty-ferret-ale-500ml £3.30 / ltr Alcohol by volume: 4.4%
Serving suggestion: Dark crystal malt delivers a full bodied biscuity taste, rounded off with a subtle floral and lemon hop aroma
Alcoholic content: 2.2 units
https://www.sainsburys.co.uk/shop/gb/groceries/product/details/all-beers/birra-moretti-12x330ml £3.79 / ltr Alcohol by volume: 4.6%
Alcoholic content: 1.5 units
https://www.sainsburys.co.uk/shop/gb/groceries/product/details/all-beers/birra-moretti-lager-4x330ml £3.98 / ltr Alcohol by volume: 4.6%
Alcoholic content: 1.5 units
https://www.sainsburys.co.uk/shop/gb/groceries/product/details/all-beers/birra-moretti-lager-660ml £2.88 / ltr Alcohol by volume: 4.6%
https://www.sainsburys.co.uk/shop/gb/groceries/product/details/all-beers/birra-moretti-lager-beer-bottle-18x330ml £2.86 / ltr Alcohol by volume: 4.6%
https://www.sainsburys.co.uk/shop/gb/groceries/product/details/all-beers/bishops-finger-a

In [85]:
# Cleanup

import pandas as pd
import re

df = pd.DataFrame(data=data, columns=['href','ppl','abv_raw'])

abv_raw_list  = df['abv_raw'].to_list()
abv_list = []

for i in abv_raw_list:
    m = re.findall('([0-9]*\.?[0-9]*)\s*%',i)
    if len(m) > 0:
        abv_list.append(list(m)[0])
    else:
        abv_list.append('NaN')


# Conver ppl values into workable numbers

df['ppl_int']= pd.to_numeric(df['ppl'].apply(lambda x: x[1:5])) 

df['price_per_unit'] = df['ppl_int']/pd.to_numeric(df['abv'])

# Extract beer names

df['beer_name'] = df['href'].apply(lambda x: x.replace('https://www.sainsburys.co.uk/shop/gb/groceries/product/details/all-beers/',''))

df.sort_values(by='price_per_unit')


In [119]:
df.to_csv('beers.csv')

In [16]:
import plotly as plt
import pandas as pd
df = pd.read_csv('beers.csv')
df = df[df['price_per_unit'] < 5]

In [17]:
df

,Unnamed: 0,href,ppl,abv_raw,abv,ppl_int,price_per_unit,beer_name
0,0,https://www.sainsburys.co.uk/shop/gb/groceries...,£3.30 / ltr,Alcohol by volume: 4.4%\nServing suggestion: D...,4.4,3.30,0.750000,badger-fursty-ferret-ale-500ml
1,1,https://www.sainsburys.co.uk/shop/gb/groceries...,£3.79 / ltr,Alcohol by volume: 4.6%\nAlcoholic content: 1....,4.6,3.79,0.823913,birra-moretti-12x330ml
2,2,https://www.sainsburys.co.uk/shop/gb/groceries...,£3.98 / ltr,Alcohol by volume: 4.6%\nAlcoholic content: 1....,4.6,3.98,0.865217,birra-moretti-lager-4x330ml
3,3,https://www.sainsburys.co.uk/shop/gb/groceries...,£2.88 / ltr,Alcohol by volume: 4.6%,4.6,2.88,0.626087,birra-moretti-lager-660ml
4,4,https://www.sainsburys.co.uk/shop/gb/groceries...,£2.86 / ltr,Alcohol by volume: 4.6%,4.6,2.86,0.621739,birra-moretti-lager-beer-bottle-18x330ml
5,5,https://www.sainsburys.co.uk/shop/gb/groceries...,£2.60 / ltr,Alcohol by volume: 5.4%\nAlcoholic content: 2....,5.4,2.60,0.481481,bishops-finger-ale-500ml
6,6,https://www.sainsburys.co.uk/shop/gb/groceries...,£3.00 / ltr,Alcohol by volume: 4.4%\nServing suggestion: A...,4.4,3.00,0.681818,black-sheep-ale--bottle-500ml
7,7,https://www.sainsburys.co.uk/shop/gb/groceries...,£4.17 / ltr,Alcohol by volume: 5.4%,5.4,4.17,0.772222,brewdog-punk-india-pale-ale-ipa-4x330ml
8,8,https://www.sainsburys.co.uk/shop/gb/groceries...,£1.77 / ltr,Alcohol by volume: 4.5%,4.5,1.77,0.393333,budweiser-18x440ml
9,9,https://www.sainsburys.co.uk/shop/gb/groceries...,£1.83 / ltr,Alcohol by volume: 4.5%\nAlcoholic content: 1....,4.5,1.83,0.406667,budweiser-lager-beer-bottles-20x300ml


In [30]:
import plotly.express as px
fig = px.scatter(df, x="price_per_unit", 
y="abv", 
hover_data=['beer_name'],
labels={
                     "price_per_unit": "Price per Unit of Alcohol (£)",
                     "abv": "Alcohol Volume (%)",
                     "ppl_int": "Price per Litre (£)"
                 },
color='ppl_int',
color_continuous_scale = 'viridis',
template='plotly_dark',
title="Sainsbury's Beer Price per Unit %Vol Distributions")
fig.update_traces(marker_size=10)

fig.show()

,beer_name,price_per_unit
0,badger-fursty-ferret-ale-500ml,0.750000
1,birra-moretti-12x330ml,0.823913
2,birra-moretti-lager-4x330ml,0.865217
3,birra-moretti-lager-660ml,0.626087
4,birra-moretti-lager-beer-bottle-18x330ml,0.621739
5,bishops-finger-ale-500ml,0.481481
6,black-sheep-ale--bottle-500ml,0.681818
7,brewdog-punk-india-pale-ale-ipa-4x330ml,0.772222
8,budweiser-18x440ml,0.393333
9,budweiser-lager-beer-bottles-20x300ml,0.406667
